# A study of Hare and Hounds

Hare and Hounds is a two-player game that recently became popular (again) with its release in the Nintendo Switch _Clubhouse Games_ collection.


Hare games were actually popular in medieval Northern Europe, according to [Wikipedia](https://en.wikipedia.org/wiki/Hare_games).

This is the game board and initial setup:

![Game Board](https://upload.wikimedia.org/wikipedia/commons/8/85/Hare_and_Hounds_board.png)

A player controls the hounds, the other controls the hare. Players take turns making a move.
Hounds move only from left to right (not backwards), while the hare may move in any direction.
Neither side may capture, a move is only legal if the destination square is free.
The hare wins if it reaches the leftmost square.
Hounds win if the hare is stalemated (i.e. if it is the hare's turn and there are no legal moves).
If 30 moves elapse before either player wins the game, the hare wins.

There has been a lot of debate on online forums about who has a winning strategy.

The game looks simple enough to be exaustively searched, the number of legal positions is only:

$$
\binom{11}{3} \times 8 \times 2 = 2640
$$

Where the factors of the left-hand side are, resepectively, the number of ways to place three hounds, the number of ways to place a hare,
and two choices for whose turn it is.

In our notation, positions are numbered from 1 to 11, where the topmost row is 1-3, left to right, the central row is 4-8, left to right, and the bottom row is 9-11, left to right.

Some imports:

In [1]:
from enum import IntEnum
from dataclasses import dataclass, field
from functools import cache, total_ordering, reduce

First, the board geometry:

In [2]:
connectivity = {
    1: [2, 4, 5, 6],
    2: [1, 3, 6],
    3: [2, 6, 7, 8],
    4: [1, 5, 9],
    5: [1, 4, 6, 9],
    6: [1, 2, 3, 5, 7, 9, 10, 11],
    7: [3, 6, 8, 11],
    8: [3, 7, 11],
    9: [4, 5, 6, 10],
    10: [6, 9, 11],
    11: [6, 7, 8, 10]
}

forward_connectivity = {
    1: [2, 5, 6],
    2: [3, 6],
    3: [7, 8],
    4: [1, 5, 9],
    5: [1, 6, 9],
    6: [2, 3, 7, 10, 11],
    7: [3, 8, 11],
    8: [],
    9: [5, 6, 10],
    10: [6, 11],
    11: [7, 8]
}

Then, we define a position evaluation. The evaluation is has two parts: who wins, and in how many moves.
By convention, the hound will be the minimizing player and the hare the maximizing player.

Evaluations are totally ordered. A perfect player prefers:
- winning to losing (in any number of moves)
- winning in fewer moves to winning in more moves (i.e. winning as fast as possible)
- losing in more moves to losing in fewer moves (i.e. surviving as long as possible, playing "in the most critical way" as chess players would put it.)

In [3]:
class Player(IntEnum):
    HOUND = 1
    HARE = 2


@dataclass(slots=True, frozen=True)
@total_ordering
class PositionEval:
    winner: Player
    moves: int = 0
    principal_variation: list[tuple[int, int]] = field(default_factory=list)

    def __lt__(self, other):
        match self.winner, other.winner:
            case Player.HOUND, Player.HARE : return True
            case Player.HARE , Player.HOUND: return False
            case Player.HOUND, Player.HOUND: return self.moves < other.moves
            case Player.HARE , Player.HARE : return self.moves > other.moves

    __add__ = lambda s, o: PositionEval(s.winner, s.moves + 1, ([o] if o else []) + s.principal_variation)
    __radd__ = __add__
    __repr__ = lambda s: f'{s.winner.name} wins in {s.moves} moves'

Now we define a position and its "children", i.e. the position reachable with a legal move from that position.

A position is completely defined by the position of the hounds, which we represent as a sorted tuple, the position of the hare, and the player whose turn it is.

In [4]:
@dataclass(slots=True, frozen=True)
class Position:
    hounds: tuple[int, int, int]
    hare: int
    turn: Player

    def get_children(self):
        if self.turn == Player.HOUND:
            for hound in self.hounds:
                for v in forward_connectivity[hound]:
                    if v not in self.hounds and v != self.hare:
                        yield (hound, v), Position(
                            tuple(sorted(set(self.hounds) - {hound} | {v})),
                            self.hare,
                            Player.HARE
                        )
        elif self.turn == Player.HARE:
            for v in connectivity[self.hare]:
                if v not in self.hounds:
                    yield (self.hare, v), Position(self.hounds, v, Player.HOUND)

    def move(self, *args):
        def _move_1(s, m):
            for move, child in s.get_children():
                if m == move:
                    return child
            raise Exception('Illegal move')
        return reduce(_move_1, args, self)

Evaluating who wins is now just the minimax algorithm.

In [5]:
@cache
def eval_position(p: Position, plies: int = 60):
    if p.hare == 4 or plies == 0:
        return PositionEval(winner=Player.HARE)

    children_evals = ((eval_position(child, plies-1), move) for move, child in p.get_children())
    if p.turn == Player.HARE:
        ev, move = max(children_evals, default=(None, PositionEval(winner=Player.HOUND, moves=-1)))
    elif p.turn == Player.HOUND:
        ev, move = min(children_evals, default=(None, PositionEval(winner=Player.HARE, moves=-1)))
    return ev + move

## So, who wins?

It turns out the hounds win in the starting position regardless of who starts, in 23 plies if the hounds start, and in 24 plies if the hare starts.

In [6]:
eval_position(Position(hounds=(1, 4, 9), hare=8, turn=Player.HOUND))

HOUND wins in 23 moves

In [7]:
eval_position(Position(hounds=(1, 4, 9), hare=8, turn=Player.HARE))

HOUND wins in 24 moves

As a matter of fact, the hounds win regardless of where the hare is placed and regardless of who starts:

In [8]:
{
    (j, k): eval_position(Position(hounds=(1, 4, 9), hare=j, turn=k))
    for j in set(range(1, 12)) - {1, 4, 9}
    for k in (Player.HOUND, Player.HARE)
}

{(2, <Player.HOUND: 1>): HOUND wins in 23 moves,
 (2, <Player.HARE: 2>): HOUND wins in 24 moves,
 (3, <Player.HOUND: 1>): HOUND wins in 23 moves,
 (3, <Player.HARE: 2>): HOUND wins in 24 moves,
 (5, <Player.HOUND: 1>): HOUND wins in 27 moves,
 (5, <Player.HARE: 2>): HOUND wins in 24 moves,
 (6, <Player.HOUND: 1>): HOUND wins in 23 moves,
 (6, <Player.HARE: 2>): HOUND wins in 28 moves,
 (7, <Player.HOUND: 1>): HOUND wins in 23 moves,
 (7, <Player.HARE: 2>): HOUND wins in 24 moves,
 (8, <Player.HOUND: 1>): HOUND wins in 23 moves,
 (8, <Player.HARE: 2>): HOUND wins in 24 moves,
 (10, <Player.HOUND: 1>): HOUND wins in 23 moves,
 (10, <Player.HARE: 2>): HOUND wins in 24 moves,
 (11, <Player.HOUND: 1>): HOUND wins in 23 moves,
 (11, <Player.HARE: 2>): HOUND wins in 24 moves}

## This game has opposition too?

This position is _zugzwang_ (i.e. whoever is to move, loses!)

In [9]:
eval_position(Position(hounds=(1, 6, 9), hare=2, turn=Player.HOUND))

HARE wins in 60 moves

In [10]:
eval_position(Position(hounds=(1, 6, 9), hare=2, turn=Player.HARE))

HOUND wins in 18 moves

## The perfect game

We can output the principal variation from the opening position, this results in the _perfect_ game: the hounds, who have a winning strategy, win in the fewest moves possible, and the hare tries to survive as long as it can.

In [11]:
e = eval_position(Position(hounds=(1, 4, 9), hare=8, turn=Player.HOUND))
list(e.principal_variation)

[(1, 6),
 (8, 11),
 (4, 5),
 (11, 10),
 (5, 1),
 (10, 11),
 (1, 2),
 (11, 8),
 (6, 11),
 (8, 7),
 (2, 6),
 (7, 3),
 (9, 5),
 (3, 7),
 (5, 1),
 (7, 3),
 (1, 2),
 (3, 8),
 (6, 3),
 (8, 7),
 (2, 6),
 (7, 8),
 (6, 7)]

Interestingly, after `(1, 6), (8, 11), (4, 5), (11, 10), (5, 1)` we are _exactly_ in the zugzwang position analyzed above! The hare is on the other side, but the position is obviously symmetrical.

In [12]:
Position(hounds=(1, 4, 9), hare=8, turn=Player.HOUND).move(*e.principal_variation[:5])

Position(hounds=(1, 6, 9), hare=10, turn=<Player.HARE: 2>)

## The rare hounds stalemate!
There exists a position where the hounds are stalemated.

In [13]:
stalemate = Position(hounds=(7, 8, 11), hare=3, turn=Player.HOUND)
assert not list(stalemate.get_children())

An interesting rules question is how to deal with this position. It's not very relevant for perfect play as the players would have to collaborate to get there, and this program adjudicates it as a win for the hare player, by "symmetry" with the "stalemate is a loss" rule applied to the hare. Stalemate for either player is therefore a loss.